# Rocket Bunny Dashboard Prototype

FILENAME: rb_visuals.ipynb

DATE CREATED: 10-APR-21

## PHASE 1: Environment Setup

In [27]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

Function declaration

In [28]:
def is_ma_pos(val):
    
    if val > 0:
        return True
    else:
        return False

In [29]:
def numpy_ewma_vectorized(data, window):

    alpha = 2 /(window + 1.0)
    alpha_rev = 1-alpha

    scale = 1/alpha_rev
    n = data.shape[0]

    r = np.arange(n)
    scale_arr = scale**r
    offset = data[0]*alpha_rev**(r+1)
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]
    return out

In [30]:
def rma(x, n, y0):
        a = (n-1) / n
        ak = a**np.arange(len(x)-1, -1, -1)
        return np.r_[np.full(n, np.nan), y0, np.cumsum(ak * x) / ak / n + y0 * a**np.arange(1, len(x)+1)]

Class definition

## PHASE 2: Data ETL

Read in the raw data

In [77]:


raw_data = pd.read_csv("data/bunny-usd-max.csv")
#data = data.query("type == 'conventional' and region == 'Albany'")
#raw_data["date"] = pd.to_datetime(raw_data["date"], format="%Y-%m-%d")

#data.sort_values("Date", inplace=True)

raw_data.head(5)

,snapped_at,price,market_cap,total_volume
0,2021-03-12 00:00:00 UTC,3.544890e-11,0.0,1.269872e+07
1,2021-03-13 00:00:00 UTC,3.544890e-11,0.0,1.269872e+07
2,2021-03-14 00:00:00 UTC,2.723985e-11,0.0,4.711907e+06
3,2021-03-15 00:00:00 UTC,2.276712e-11,0.0,1.814948e+06
4,2021-03-16 00:00:00 UTC,3.197680e-11,0.0,1.701247e+06


In [78]:
app = dash.Dash(__name__)

In [79]:

data = raw_data.rename({'snapped_at': 'dtg'}, axis=1)
data.dtypes

dtg              object
price           float64
market_cap      float64
total_volume    float64
dtype: object

In [80]:
col_list = list(data.columns)
col_list

['dtg', 'price', 'market_cap', 'total_volume']

In [81]:
close_np = data['price'].to_list()
close_np

[3.544890372449417e-11,
 3.544890372449417e-11,
 2.7239851621885825e-11,
 2.276711980520152e-11,
 3.1976800418536546e-11,
 2.8077474254547858e-11,
 2.1126616432418585e-11,
 2.0963108885770787e-11,
 1.7818568348394942e-11,
 1.714732901367917e-11,
 1.2481281902033133e-11,
 1.2533681483670114e-11,
 1.1875716146667161e-11,
 9.457534146178276e-12,
 7.305428315404949e-12,
 8.744554690166982e-12,
 7.207941603822242e-12,
 7.650775307094981e-12,
 9.388180178763074e-12,
 7.174640793551023e-12,
 8.115413911027787e-12,
 7.999102012443927e-12,
 8.651487749048858e-12]

In [97]:
data['date'] = pd.to_datetime(data['dtg']).dt.date
data['year'] = pd.DatetimeIndex(data['date']).year
data['month'] = pd.DatetimeIndex(data['date']).month
data['day'] = pd.DatetimeIndex(data['date']).day
data['hour'] = pd.DatetimeIndex(data['date']).hour.astype(str)

In [98]:
data

,dtg,price,market_cap,total_volume,close_diff,bool_close_diff,gain,loss,avg_gain,avg_loss,...,SMA-MACD,SMA-MACD-ratio,bool_diff_SMA7_SMA30,EMA12,EMA26,date,year,month,day,hour
0,2021-03-12 00:00:00 UTC,3.544890e-11,0.0,1.269872e+07,NaN,False,NaN,NaN,NaN,NaN,...,NaN,NaN,False,3.544890e-11,3.544890e-11,2021-03-12,2021,3,12,0
1,2021-03-13 00:00:00 UTC,3.544890e-11,0.0,1.269872e+07,0.000000e+00,False,0.000000e+00,-0.000000e+00,NaN,NaN,...,NaN,NaN,False,3.544890e-11,3.544890e-11,2021-03-13,2021,3,13,0
2,2021-03-14 00:00:00 UTC,2.723985e-11,0.0,4.711907e+06,-8.209052e-12,False,0.000000e+00,8.209052e-12,NaN,NaN,...,NaN,NaN,False,3.418597e-11,3.418597e-11,2021-03-14,2021,3,14,0
3,2021-03-15 00:00:00 UTC,2.276712e-11,0.0,1.814948e+06,-4.472732e-12,False,0.000000e+00,4.472732e-12,NaN,NaN,...,NaN,NaN,False,3.242923e-11,3.242923e-11,2021-03-15,2021,3,15,0
4,2021-03-16 00:00:00 UTC,3.197680e-11,0.0,1.701247e+06,9.209681e-12,True,9.209681e-12,0.000000e+00,NaN,NaN,...,NaN,NaN,False,3.235962e-11,3.235962e-11,2021-03-16,2021,3,16,0
5,2021-03-17 00:00:00 UTC,2.807747e-11,0.0,1.015236e+06,-3.899326e-12,False,0.000000e+00,3.899326e-12,NaN,NaN,...,NaN,NaN,False,3.170083e-11,3.170083e-11,2021-03-17,2021,3,17,0
6,2021-03-18 00:00:00 UTC,2.112662e-11,0.0,7.644860e+05,-6.950858e-12,False,0.000000e+00,6.950858e-12,NaN,NaN,...,NaN,NaN,False,3.007403e-11,3.007403e-11,2021-03-18,2021,3,18,0
7,2021-03-19 00:00:00 UTC,2.096311e-11,0.0,5.282036e+05,-1.635075e-13,False,0.000000e+00,1.635075e-13,NaN,NaN,...,NaN,NaN,False,2.867235e-11,2.867235e-11,2021-03-19,2021,3,19,0
8,2021-03-20 00:00:00 UTC,1.781857e-11,0.0,3.706957e+05,-3.144541e-12,False,0.000000e+00,3.144541e-12,NaN,NaN,...,NaN,NaN,False,2.700254e-11,2.700254e-11,2021-03-20,2021,3,20,0
9,2021-03-21 00:00:00 UTC,1.714733e-11,0.0,3.667799e+05,-6.712393e-13,False,0.000000e+00,6.712393e-13,NaN,NaN,...,NaN,NaN,False,2.548635e-11,2.548635e-11,2021-03-21,2021,3,21,0


Calculate 'gain' and 'loss' columns

In [99]:
data['close_diff'] = data['price'].diff()
data['bool_close_diff'] = data['close_diff'].apply(is_ma_pos)

In [100]:
data['gain'] = data.close_diff.mask(data.close_diff < 0, 0.0)
data['loss'] = -data.close_diff.mask(data.close_diff > 0, -0.0)

n=14 # rsi window number
data['avg_gain'] = rma(data.gain[n+1:].to_numpy(), n, np.nansum(data.gain.to_numpy()[:n+1])/n)
data['avg_loss'] = rma(data.loss[n+1:].to_numpy(), n, np.nansum(data.loss.to_numpy()[:n+1])/n)


Calculate rolling mean

In [101]:
data['SMA7'] = data['price'].rolling(window=7).mean()
data['SMA30'] = data['price'].rolling(window=30).mean()

Calculate rolling standard deviation

In [102]:
data['STD7'] = data['price'].rolling(window=7).std()
data['STD30'] = data['price'].rolling(window=30).std()

#data['STD99'] = self.aggregate_df['Close'].rolling(window=99).std()

Calculate rolling variance

In [103]:
data['VAR7'] = data['price'].rolling(window=7).var()
data['VAR30'] = data['price'].rolling(window=30).var()

#data['VAR99'] = self.aggregate_df['Close'].rolling(window=99).var()

Calculate MACD

In [104]:
data['SMA-MACD'] = data['SMA7'] - data['SMA30']
data['SMA-MACD-ratio'] = data['SMA7'] / data['SMA30']
data['bool_diff_SMA7_SMA30'] = data['SMA-MACD'].apply(is_ma_pos)

Calculate EMA

In [105]:
window = 12

In [106]:
ema_data = data['price'].to_numpy()
        
alpha = 2 /(window + 1.0)
alpha_rev = 1-alpha

scale = 1/alpha_rev
n = ema_data.shape[0]

r = np.arange(n)
scale_arr = scale**r
offset = ema_data[0]*alpha_rev**(r+1)
pw0 = alpha*alpha_rev**(n-1)

mult = ema_data*pw0*scale_arr
cumsums = mult.cumsum()
out = offset + cumsums*scale_arr[::-1]

se = pd.Series(out)
data['EMA12'] = se.values

In [107]:
window2 = 26

In [108]:
ema_data = data['price'].to_numpy()
        
alpha = 2 /(window + 1.0)
alpha_rev = 1-alpha

scale = 1/alpha_rev
n = ema_data.shape[0]

r = np.arange(n)
scale_arr = scale**r
offset = ema_data[0]*alpha_rev**(r+1)
pw0 = alpha*alpha_rev**(n-1)

mult = ema_data*pw0*scale_arr
cumsums = mult.cumsum()
out = offset + cumsums*scale_arr[::-1]

se = pd.Series(out)
data['EMA26'] = se.values

Display the finalized dataframe

In [109]:
data.head(5)

,dtg,price,market_cap,total_volume,close_diff,bool_close_diff,gain,loss,avg_gain,avg_loss,...,SMA-MACD,SMA-MACD-ratio,bool_diff_SMA7_SMA30,EMA12,EMA26,date,year,month,day,hour
0,2021-03-12 00:00:00 UTC,3.544890e-11,0.0,1.269872e+07,NaN,False,NaN,NaN,NaN,NaN,...,NaN,NaN,False,3.544890e-11,3.544890e-11,2021-03-12,2021,3,12,0
1,2021-03-13 00:00:00 UTC,3.544890e-11,0.0,1.269872e+07,0.000000e+00,False,0.000000e+00,-0.000000e+00,NaN,NaN,...,NaN,NaN,False,3.544890e-11,3.544890e-11,2021-03-13,2021,3,13,0
2,2021-03-14 00:00:00 UTC,2.723985e-11,0.0,4.711907e+06,-8.209052e-12,False,0.000000e+00,8.209052e-12,NaN,NaN,...,NaN,NaN,False,3.418597e-11,3.418597e-11,2021-03-14,2021,3,14,0
3,2021-03-15 00:00:00 UTC,2.276712e-11,0.0,1.814948e+06,-4.472732e-12,False,0.000000e+00,4.472732e-12,NaN,NaN,...,NaN,NaN,False,3.242923e-11,3.242923e-11,2021-03-15,2021,3,15,0
4,2021-03-16 00:00:00 UTC,3.197680e-11,0.0,1.701247e+06,9.209681e-12,True,9.209681e-12,0.000000e+00,NaN,NaN,...,NaN,NaN,False,3.235962e-11,3.235962e-11,2021-03-16,2021,3,16,0


In [110]:
col_list = list(data.columns)
col_list

['dtg',
 'price',
 'market_cap',
 'total_volume',
 'close_diff',
 'bool_close_diff',
 'gain',
 'loss',
 'avg_gain',
 'avg_loss',
 'SMA7',
 'SMA30',
 'STD7',
 'STD30',
 'VAR7',
 'VAR30',
 'SMA-MACD',
 'SMA-MACD-ratio',
 'bool_diff_SMA7_SMA30',
 'EMA12',
 'EMA26',
 'date',
 'year',
 'month',
 'day',
 'hour']

## PHASE 3: Data Visualization

Simple Moving Average (SMA) Visualization

In [116]:
sma_fig = go.Figure()
sma_fig.add_trace(go.Scatter(x=data['date'], y=data['price'],
                    mode='lines',
                    name='Price Timeseries'))
sma_fig.add_trace(go.Scatter(x=data['date'], y=data['SMA7'],
                    mode='lines',
                    name='Timeseries SMA7'))
sma_fig.add_trace(go.Scatter(x=data['date'], y=data['SMA30'],
                    mode='lines', name='Timeseries SMA30'))

sma_fig.update_layout(
    title="Rocket Bunny Close Price and Simple Moving Average",
    xaxis_title="Date",
    yaxis_title="Price",
    legend_title="Legend Title",
    font=dict(
        family="Helvetica",
        size=13,
        color="Black"
    )
)
sma_fig.show()

Gain / Loss visualization 

In [117]:
gain_loss_fig = go.Figure()
gain_loss_fig.add_trace(go.Scatter(x=data['date'], y=data['gain'],
                    mode='lines',
                    name='Daily Gain Timeseries'))
gain_loss_fig.add_trace(go.Scatter(x=data['date'], y=data['loss'],
                    mode='lines',
                    name='Daily Loss Timeseries'))

gain_loss_fig.update_layout(
    title="Rocket Bunny Daily Change in Gain & Loss",
    xaxis_title="Date",
    yaxis_title="Price",
    legend_title="Legend Title",
    font=dict(
        family="Helvetica",
        size=13,
        color="Black"
    )
)
gain_loss_fig.show()